**Scrape Spotrac Data**

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def findnth(string, substring, n):
    parts = string.split(substring, n + 1)
    if len(parts) <= n + 1:
        return -1
    return len(string) - len(parts[-1]) - len(substring)

In [3]:
def get_player_link(var):
    player_link = var[findnth(var, 'h', 1):findnth(var, '/', 5)+1]
    return player_link

In [4]:
def get_player_name(var):
    player_name = var[findnth(var, '>', 0)+1:findnth(var, '<', 1)]
    return player_name

In [5]:
df = pd.DataFrame()

In [6]:
teams = ['https://www.spotrac.com/nfl/arizona-cardinals/cap/2021/', 'https://www.spotrac.com/nfl/atlanta-falcons/cap/2021/', 'https://www.spotrac.com/nfl/baltimore-ravens/cap/2021/', 'https://www.spotrac.com/nfl/buffalo-bills/cap/2021/', 'https://www.spotrac.com/nfl/carolina-panthers/cap/2021/', 'https://www.spotrac.com/nfl/chicago-bears/cap/2021/', 'https://www.spotrac.com/nfl/cincinnati-bengals/cap/2021/', 'https://www.spotrac.com/nfl/cleveland-browns/cap/2021/', 'https://www.spotrac.com/nfl/dallas-cowboys/cap/2021/', 'https://www.spotrac.com/nfl/denver-broncos/cap/2021/', 'https://www.spotrac.com/nfl/detroit-lions/cap/2021/', 'https://www.spotrac.com/nfl/green-bay-packers/cap/2021/', 'https://www.spotrac.com/nfl/houston-texans/cap/2021/', 'https://www.spotrac.com/nfl/indianapolis-colts/cap/2021/', 'https://www.spotrac.com/nfl/jacksonville-jaguars/cap/2021/', 'https://www.spotrac.com/nfl/kansas-city-chiefs/cap/2021/', 'https://www.spotrac.com/nfl/las-vegas-raiders/cap/2021/', 'https://www.spotrac.com/nfl/los-angeles-chargers/cap/2021/', 'https://www.spotrac.com/nfl/los-angeles-rams/cap/2021/', 'https://www.spotrac.com/nfl/miami-dolphins/cap/2021/', 'https://www.spotrac.com/nfl/minnesota-vikings/cap/2021/', 'https://www.spotrac.com/nfl/new-england-patriots/cap/2021/', 'https://www.spotrac.com/nfl/new-orleans-saints/cap/2021/', 'https://www.spotrac.com/nfl/new-york-giants/cap/2021/', 'https://www.spotrac.com/nfl/new-york-jets/cap/2021/', 'https://www.spotrac.com/nfl/philadelphia-eagles/cap/2021/', 'https://www.spotrac.com/nfl/pittsburgh-steelers/cap/2021/', 'https://www.spotrac.com/nfl/san-francisco-49ers/cap/2021/', 'https://www.spotrac.com/nfl/seattle-seahawks/cap/2021/', 'https://www.spotrac.com/nfl/tampa-bay-buccaneers/cap/2021/', 'https://www.spotrac.com/nfl/tennessee-titans/cap/2021/', 'https://www.spotrac.com/nfl/washington-football-team/cap/2021/']

In [7]:
for team in teams:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    options = Options()
    options.add_argument('--disable-blink-features=AutomationControlled')
    driver.get(team)
    soup = BeautifulSoup(driver.page_source)
    driver.quit()
    
    table = soup.find_all('table')[0]
    players = table.find_all("a")
    players = [str(player) for player in players]
    players = [player for player in players if 'https' in player]
    
    for player in players:
    
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        options = Options()
        options.add_argument('--disable-blink-features=AutomationControlled')
        driver.get(get_player_link(player))
        

        popup = driver.find_element('xpath', '//*[@id="PopupSignupForm_0"]/div[2]/div[1]')
        popup.click()

        button = driver.find_element("xpath", '//*[@id="cash-earnings"]')
        button.click()

        get_url = driver.current_url

        response = urlopen(get_url)
        html = response.read()

        soup = BeautifulSoup(html)
        driver.quit()

        earnings_table = soup.find('table', {'class': 'earningstable rtable'})
        tab_data = [[celldata.text for celldata in rowdata.find_all(["th","td"])]
                            for rowdata in earnings_table.find_all("tr")]
        earnings_table_rows = earnings_table.find_all('tr')
        team_list = []
        for row in earnings_table_rows[1:]:
            img = row.find('img')
            try:
                team_list.append(img['src'].split('.png')[0].rsplit('/', 1)[1].upper())
            except:
                team_list.append('')

        player_df = pd.DataFrame(tab_data[1:], columns = tab_data[0])
        player_df.insert(loc=0, column='Player', value=get_player_name(player))
        
        player_df['Team'] = team_list
        player_df = player_df.apply(lambda x: x.str.replace(',', ''))
        player_df = player_df.apply(lambda x: x.str.replace('$', ''))

        player_df['Year'] = pd.to_numeric(player_df['Year'],errors='coerce')
        player_df = player_df.dropna()
        player_df['Year'] = player_df['Year'].astype('int64')

        player_df[['Year', 'Salary', 'Signing Bonus', 'Roster Bonus', 'Workout Bonus', 'Restructure Bonus', 'Option Bonus', 'Incentive', 'Earnings']] = player_df[['Year', 'Salary', 'Signing Bonus', 'Roster Bonus', 'Workout Bonus', 'Restructure Bonus', 'Option Bonus', 'Incentive', 'Earnings']].apply(pd.to_numeric, errors='ignore')
        df = df.append(player_df)

In [ ]:
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#options = Options()
#options.add_argument('--disable-blink-features=AutomationControlled')
#driver.get('https://www.spotrac.com/nfl/buffalo-bills/cap/2021/')
#soup = BeautifulSoup(driver.page_source)
#driver.quit()

In [ ]:
#table = soup.find_all('table')[0]
#players = table.find_all("a")
#players = [str(player) for player in players]
#players = [player for player in players if 'https' in player]

In [ ]:
#for player in players:
    
    #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    #options = Options()
    #options.add_argument('--disable-blink-features=AutomationControlled')
    #driver.get(get_player_link(player))
    #driver.get(players_links[0])

    #popup = driver.find_element('xpath', '//*[@id="PopupSignupForm_0"]/div[2]/div[1]')
    #popup.click()

    #button = driver.find_element("xpath", '//*[@id="cash-earnings"]')
    #button.click()

    #get_url = driver.current_url

#    response = urlopen(get_url)
#    html = response.read()
#
#    soup = BeautifulSoup(html)
#    driver.quit()
#    
#    earnings_table = soup.find('table', {'class': 'earningstable rtable'})
#    tab_data = [[celldata.text for celldata in rowdata.find_all(["th","td"])]
#                        for rowdata in earnings_table.find_all("tr")]
#    earnings_table_rows = earnings_table.find_all('tr')
#    team_list = []
#    for row in earnings_table_rows[1:]:
#        img = row.find('img')
#        try:
#            team_list.append(img['src'].split('.png')[0].rsplit('/', 1)[1].upper())
#        except:
#            team_list.append('')
#            
#    player_df = pd.DataFrame(tab_data[1:], columns = tab_data[0])
#    player_df.insert(loc=0, column='Player', value=get_player_name(player))
    #df.insert(loc=0, column='Player', value=players_names[0])
#    player_df['Team'] = team_list
#    player_df = player_df.apply(lambda x: x.str.replace(',', ''))
#    player_df = player_df.apply(lambda x: x.str.replace('$', ''))

#    player_df['Year'] = pd.to_numeric(player_df['Year'],errors='coerce')
#    player_df = player_df.dropna()
#    player_df['Year'] = player_df['Year'].astype('int64')

#    player_df[['Year', 'Salary', 'Signing Bonus', 'Roster Bonus', 'Workout Bonus', 'Restructure Bonus', 'Option Bonus', 'Incentive', 'Earnings']] = player_df[['Year', 'Salary', 'Signing Bonus', 'Roster Bonus', 'Workout Bonus', 'Restructure Bonus', 'Option Bonus', 'Incentive', 'Earnings']].apply(pd.to_numeric, errors='ignore')
#    df = df.append(player_df)

In [8]:
df = df.reset_index()

In [15]:
df['Team'] = df['Team'].apply(lambda x: x.replace('_', ''))
df['Team'] = df['Team'].apply(lambda x: x.replace('0', ''))
df['Team'] = df['Team'].apply(lambda x: x.replace('1', ''))
df['Team'] = df['Team'].apply(lambda x: x.replace('2', ''))
df['Team'] = df['Team'].apply(lambda x: x.replace('3', ''))

In [17]:
df = df.drop(['index'], axis=1)

In [18]:
df

,Player,Year,Team,Awards,Salary,Signing Bonus,Roster Bonus,Workout Bonus,Restructure Bonus,Option Bonus,Incentive,Earnings
0,Chandler Jones,2012,NE,,390000,4384364,-,-,-,-,-,4774364
1,Chandler Jones,2013,NE,,761522,-,-,-,-,-,-,761522
2,Chandler Jones,2014,NE,,1133045,-,-,-,-,-,-,1133045
3,Chandler Jones,2015,NE,,1504568,-,-,-,-,-,-,1504568
4,Chandler Jones,2016,ARI,,7799000,-,-,-,-,-,-,7799000
...,...,...,...,...,...,...,...,...,...,...,...,...
12777,Jeremy Reaves,2021,WAS,,296821,-,-,-,-,-,-,296821
12778,Jeremy Reaves,2022,WAS,,965000,-,-,-,-,-,-,965000
12779,Will Bradley-King,2021,WAS,,211332,100100,-,-,-,-,-,311432
12780,Will Bradley-King,2021,WAS,,36666,-,-,-,-,-,-,36666


In [19]:
df.to_csv(r'C:\Users\FIDataSci.DESKTOP-5QIEL80\spotrac-dataframe-csv', sep='\t', encoding='utf-8')

find nth function : https://www.tutorialspoint.com/How-to-find-the-nth-occurrence-of-substring-in-a-string-in-Python#:~:text=You%20can%20find%20the%20nth,occurs%20more%20than%20n%20times.